In [1]:
"""
Class
- Normalize Data
- Harmonize Data
- Build Classifier
- Classify
- Split
- - Class
"""

'\nClass\n- Normalize Data\n- Harmonize Data\n- Build Classifier\n- Classify\n- Split\n- - Class\n'

In [2]:
# from fish_helpers import *
# import torch
# import scvi
# import scanpy as sc
# import anndata
# sc.set_figure_params(figsize=(7, 7))
import sys
sys.path.insert(0, '/home/fangming/projects/dredfish/packages/PySpots/')
sys.path.insert(0, '/home/fangming/projects/myutils/')

from fish_helpers import *
# import torch
# import scvi
import scanpy as sc
import anndata

import basicu
sc.set_figure_params(figsize=(7, 7))

In [3]:
ref_path = '/bigstore/binfo/mouse/Brain/DRedFISH/Allen_V3_Reference/10X_dpnmf/'
weights = pd.read_csv(os.path.join(ref_path,'weights.csv'),index_col=0)
# vizgen_distributions = pd.read_csv('/bigstore/binfo/mouse/Brain/FISH/Vizgen_data/cell_type_abundances_S2.csv',index_col=0)

In [4]:
""" Data """
""" Load Measured Data """
# # data = anndata.read_h5ad('/bigstore/Images2021/gaby/dredFISH/DPNMF_PolyA_2021Nov19/results/clustered.h5ad')
# measured_path = '/bigstore/Images2021/gaby/dredFISH/DPNMF_PolyA_2021Nov19/results'
# # if os.path.exists(os.path.join(measured_path,'clustered.h5ad')):
# measured_data = anndata.read_h5ad(os.path.join(measured_path,'clustered.h5ad'))
# measured_data.obs_names_make_unique()
# # else:
#     measured_counts = np.load(os.path.join(measured_path,'vectors.npy'),allow_pickle=True)
#     measured_cell_metadata = pd.read_csv(os.path.join(measured_path,'cell_metadata.csv'),low_memory=False,index_col=0)
#     measured_cells = np.array(measured_cell_metadata.index)
#     measured_genes = np.array(weights.columns)
#     measured_data = anndata.AnnData(X=measured_counts,var=pd.DataFrame(index=measured_genes),obs=pd.DataFrame(index=measured_cells))
#     for column in measured_cell_metadata.columns:
#         if measured_cell_metadata[column].dtype=='O':
#             measured_data.obsm[column] = np.array(measured_cell_metadata[column].astype(str))
#         else:
#             measured_data.obsm[column] = np.array(measured_cell_metadata[column])
#     measured_data.obs_names_make_unique()
#     measured_data.write(filename=os.path.join(measured_path,'anndata.h5ad'))
#     del measured_counts,measured_cells,measured_genes,measured_cell_metadata
    
    
""" Load SmartSeq Reference """
ref_SS_path = '/bigstore/binfo/mouse/Brain/DRedFISH/Allen_V3_Reference/SmartSeq_dpnmf/'
# if os.path.exists(os.path.join(ref_SS_path,'anndata.h5ad')):
ref_SS_data = anndata.read_h5ad(os.path.join(ref_SS_path,'anndata.h5ad'))
ref_SS_data.obs_names_make_unique()
# else:
#     ref_SS_counts = np.load(os.path.join(ref_SS_path,'projected.npy'),allow_pickle=True)
#     ref_SS_cell_metadata = pd.read_csv(os.path.join(ref_SS_path,'metadata.csv'),low_memory=False,index_col=0)
#     ref_SS_cells = np.array(ref_SS_cell_metadata.index)
#     ref_SS_genes = np.array(weights.columns)
#     ref_SS_data = anndata.AnnData(X=ref_SS_counts,var=pd.DataFrame(index=ref_SS_genes),obs=pd.DataFrame(index=ref_SS_cells))
#     for column in ref_SS_cell_metadata.columns:
#         if ref_SS_cell_metadata[column].dtype=='O':
#             ref_SS_data.obsm[column] = np.array(ref_SS_cell_metadata[column].astype(str))
#         else:
#             ref_SS_data.obsm[column] = np.array(ref_SS_cell_metadata[column])
#     ref_SS_data.obs_names_make_unique()
#     ref_SS_data.write(filename=os.path.join(ref_SS_path,'anndata.h5ad'))
#     del ref_SS_counts,ref_SS_genes,ref_SS_cells,ref_SS_cell_metadata

In [5]:
ref_SS_data

AnnData object with n_obs × n_vars = 74973 × 24
    obsm: 'Level_1_class_label', 'Level_2_neighborhood_label', 'Level_3_subclass_label', 'Level_4_supertype_label', 'Level_5_cluster_label', 'cell_type_accession_color', 'cell_type_accession_id', 'cell_type_accession_label', 'cell_type_alias_color', 'cell_type_alias_id', 'cell_type_alias_label', 'cell_type_alt_alias_color', 'cell_type_alt_alias_id', 'cell_type_alt_alias_label', 'cell_type_designation_color', 'cell_type_designation_id', 'cell_type_designation_label', 'class_color', 'class_label', 'class_order', 'cluster_color', 'cluster_label', 'cluster_order', 'cortical_layer_label', 'donor_sex_label', 'exp_component_name', 'external_donor_name_color', 'external_donor_name_id', 'external_donor_name_label', 'facs_population_plan_color', 'facs_population_plan_id', 'facs_population_plan_label', 'full_genotype_color', 'full_genotype_id', 'full_genotype_label', 'injection_materials_color', 'injection_materials_id', 'injection_materials_label',

In [5]:
""" Toss Cells with no signal """
# measured_data = measured_data[measured_data.X.sum(1)>1]
print(measured_data.shape)

# remove bit 19 because of 0 IQR
selected_bits = np.array([i for i in np.arange(24) if i != 19])
measured_data = measured_data[:,selected_bits]
ref_SS_data = ref_SS_data[:,selected_bits]

# filter out cells with zero values throughout all bits
rawmat = np.clip(measured_data.layers['total_vectors'].copy(), 0, None)
bitssum = rawmat.sum(axis=1)
measured_data = measured_data[bitssum > 0]
print(measured_data.shape)
print(ref_SS_data.shape)

(97786, 24)
(97614, 23)
(74973, 23)


In [6]:
# """ Class Balance to Vizgen Distributions """
# total_cells = measured_data.shape[0]
# ref_SS_data_balanced_idx = []
# labels = ref_SS_data.obsm['Level_3_subclass_label'].astype(str)
# for label in vizgen_distributions.index:
#     idxes = np.where(labels==label)[0]
#     t_ncells = int(total_cells*vizgen_distributions.loc[label]['0'])
#     if idxes.shape[0]<t_ncells:
#         idxes = np.random.choice(idxes,t_ncells,replace=True)
#     else:
#         idxes = np.random.choice(idxes,t_ncells,replace=False)
#     ref_SS_data_balanced_idx.extend(list(idxes))
# ref_SS_data_balanced = ref_SS_data[ref_SS_data_balanced_idx,:]
# print(ref_SS_data_balanced.shape)

In [7]:

# normalization
rawmat = np.clip(measured_data.layers['total_vectors'].copy(), 0, None)
bitssum = rawmat.sum(axis=1)
rawmat = rawmat/bitssum.reshape(-1,1) #data_s.obs['total_signal'].values.reshape(-1,1)
# rawmat_p = IQR_normalize_matrix_bycol(rawmat) # warning of dividing by zero; caused by bit 19 # not used here but used in code

# store
measured_data.layers['DPNMF'] = rawmat # measured_data.layers['total_vectors'].copy()/ # *(np.array(measured_data.obs['total_signal'])/np.array(measured_data.obs['total_signal']).mean())[:,None]
# ref_SS_data_balanced.layers['DPNMF'] = ref_SS_data.X.copy() # _balanced.X.copy()
ref_SS_data.layers['DPNMF'] = ref_SS_data.X.copy()


In [8]:
(bitssum == 0).sum()


0

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

class RecursiveClassify(object):
    def __init__(self,data,ref_data,level,verbose=True):
        self.data = data
        self.ref_data = ref_data
        self.level = level
        self.verbose = verbose
        
    def run(self):
        self.normalizeData()
        self.harmonizeData()
        self.classifyData()
        self.splitData()
        
    def normalizeData(self):
        # def normalize_vector(v):
        #     v = v.copy()
        #     v = v-np.percentile(v,50)
        #     v = v/(np.percentile(v,75)-np.percentile(v,25))
        #     return v
        def zscore(v, **kwargs):
            return (v-np.mean(v, **kwargs))/(np.std(v, **kwargs))

        def zscore_matrix_bycol(mat):
            return zscore(mat, axis=0) # across rows
        
        data = np.array(self.data.layers['DPNMF'].copy())
        ref_data = np.array(self.ref_data.layers['DPNMF'].copy())
        
        
        data = zscore_matrix_bycol(data)
        ref_data = zscore_matrix_bycol(ref_data)
        
        # for i in range(data.shape[1]):
        #     """ Scale bitwise first """
        #     data[:,i] = normalize_vector(data[:,i])
        #     ref_data[:,i] = normalize_vector(ref_data[:,i])
        
        self.data.layers['DPNMF_IQR'] = data.copy()
        self.ref_data.layers['DPNMF_IQR'] = ref_data.copy()
        
    def harmonizeData(self):
        merged_data = anndata.AnnData(X=np.concatenate([self.data.layers['DPNMF_IQR'].astype(float),self.ref_data.layers['DPNMF_IQR'].astype(float)]),
                                      var=np.array(self.data.var).astype(str),
                                      obs=np.concatenate([np.array(self.data.obs.index).astype(str),np.array(self.ref_data.obs.index).astype(str)]).astype(str))
        merged_data.obs['dataset'] = np.concatenate([np.array(['a' for i in range(len(self.data.obs))]),np.array(['b' for i in range(len(self.ref_data.obs))])]).astype(str)
        merged_data.obsm['DPNMF'] = merged_data.X.copy()
        merged_data.layers['DPNMF'] = merged_data.X.copy()
        merged_data.raw = merged_data

        import scanpy.external as sce
        sce.pp.harmony_integrate(merged_data, 'dataset',basis='DPNMF',adjusted_basis='DPNMF_harmony')#,reference_values='b')
    
        self.data.obsm['DPNMF_harmony'] = np.array(merged_data[merged_data.obs['dataset'] == 'a'].obsm['DPNMF_harmony'])
        self.ref_data.obsm['DPNMF_harmony'] = np.array(merged_data[merged_data.obs['dataset'] == 'b'].obsm['DPNMF_harmony'])
        # self.data.obsm['DPNMF_harmony'] = np.array(merged_data[merged_data.obs['dataset'] == 'a'].obsm['DPNMF'])
        # self.ref_data.obsm['DPNMF_harmony'] = np.array(merged_data[merged_data.obs['dataset'] == 'b'].obsm['DPNMF'])
        
    def classBalance(self):
        """ Class Balance """
        n_cells = 100
        X = self.ref_data.obsm['DPNMF_harmony'].copy()
        Y = np.array(self.ref_data.obsm[self.level])
        Y_unique = np.unique(Y)
        X_balanced = []
        Y_balanced = []
        for y in Y_unique:
            idx = np.where(Y==y)[0]
            if idx.shape[0]>n_cells:
                idx = np.random.choice(idx,n_cells,replace=False)
            else:
                idx = np.random.choice(idx,n_cells,replace=True)
            X_balanced.append(X[idx,:])
            Y_balanced.append(Y[idx])
        self.X_balanced = np.concatenate(X_balanced)
        self.Y_balanced = np.concatenate(Y_balanced)
    
    def splitTestTrain(self):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X_balanced, self.Y_balanced, test_size=0.4, random_state=42)
        
    def buildModel(self):
        self.method = 'KNeighborsClassifier'
        self.model =  KNeighborsClassifier(15,metric='correlation')
        
    def trainModel(self):
        self.model.fit(self.X_train,self.y_train)
        
    def predictLabels(self):
        self.data.obs[self.level+'_predicted'] = self.model.predict(self.data.obsm['DPNMF_harmony'].copy())
        
    def classifyData(self):
        self.classBalance()
        self.splitTestTrain()
        self.buildModel()
        self.trainModel()
        self.predictLabels()
        
    def splitData(self):
        levels = np.array([i for i in self.ref_data.obsm if i.split('_')[0]=='Level'])
        for level in levels:
            self.ref_data.obs[level] = np.array(self.ref_data.obsm[level])
        loc = np.where(levels==self.level)[0][0]+1
        if loc>=len(levels):
            """ No More Levels"""
        else:
            next_level = levels[loc]
            self.data.obs[next_level+'_predicted'] = np.array(['Unknown' for i in range(self.data.shape[0])])
            ref_data_labels = np.array(self.ref_data.obs[self.level])
            data_labels = np.array(self.data.obs[self.level+'_predicted'])
            unique_labels = np.unique(data_labels)
            if self.verbose:
                if self.level ==levels[0]:
                    level = self.level
                else:
                    previous_level = levels[loc-2]
                    level = self.level + '_' + str(np.unique(self.ref_data.obs[previous_level])[0])
                iterable = tqdm(unique_labels,desc=level)
            else:
                iterable = unique_labels
            for label in iterable:
                try:
                    self.rc = RecursiveClassify(self.data[data_labels==label].copy(),self.ref_data[ref_data_labels==label].copy(),next_level,verbose=self.verbose)
                    self.rc.run()
                except Exception as e:
                    print(e)
                    print(label+'_Failed')
                for level in levels:
                        if level+'_predicted' in self.rc.data.obs:
                            if not level+'_predicted' in self.data.obs:
                                self.data.obs[level+'_predicted'] = np.array(['Unknown' for i in range(self.data.shape[0])])
                            try:
                                temp = np.array(self.data.obs[level+'_predicted'])
                                temp[data_labels==label] = np.array(self.rc.data.obs[level+'_predicted'])
                                self.data.obs[level+'_predicted'] = temp
                            except:
                                print(level+'_predicted '+label)
                                continue

In [ ]:
data = measured_data.copy()
ref_data  = ref_SS_data.copy()# _balanced.copy()
levels = [i for i in ref_data.obsm if i.split('_')[0]=='Level']           
rc = RecursiveClassify(data.copy(),ref_data.copy(),levels[0],verbose=True)
rc.run()    

IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


In [ ]:
rc.data

In [ ]:
sc.set_figure_params(figsize=[10,10])
levels = [i for i in rc.data.obs if i.split('_')[0]=='Level']  
for level in levels:
    rc.data.obsm[level] = np.array(rc.data.obs[level])
    sc.pl.embedding(rc.data,'stage', color=[level],palette='jet',size=5)

In [ ]:
data = rc.data

In [ ]:
levels = np.array(levels)
for i,level in enumerate(levels):
    temp2 = np.array(data.obs[levels[i]])
    if 'Other' in np.unique(data.obs[level]):
        temp = np.array([i+'_Other' for i in data.obs[levels[i-1]]])
        temp2 = np.array(data.obs[levels[i]])
        temp2[temp2=='Other'] = temp[temp2=='Other']
    data.obs[levels[i]+'_updated'] = np.array(temp2)
    data.obsm[levels[i]+'_updated'] = np.array(data.obs[levels[i]+'_updated'])
    sc.pl.embedding(data,'stage', color=['leiden',levels[i]+'_updated'],palette='jet',size=4)
    
    data_labels = np.array(data.obs[levels[i]+'_updated'])
    ref_data_labels = np.array(data.obs['opt_types'])
    data_labels_unique = np.unique(data_labels)
    ref_data_labels_unique = np.unique(ref_data_labels)
    cmat = np.zeros([data_labels_unique.shape[0],ref_data_labels_unique.shape[0]])
    for i,i_label in tqdm(enumerate(data_labels_unique),total=data_labels_unique.shape[0]):
        i_mask =data_labels==i_label
        for j,j_label in enumerate(ref_data_labels_unique):
            j_mask = ref_data_labels==j_label
            cmat[i,j] = np.sum(i_mask&j_mask)
    cmat = cmat/cmat.sum(0)[:,None].T
    cmat = pd.DataFrame(cmat,index=data_labels_unique,columns=ref_data_labels_unique)
    idx_order = []
    for idx,i in enumerate(cmat.index):
        if 'Level_5' in level:
            if i=='Unknown':
                idx_order.append(0)
            else:
                idx_order.append(int(i.split('_')[0]))
        else:
            idx_order.append(idx)
    idx_order = np.array(cmat.index)[np.argsort(np.array(idx_order))]
    column_order = []
    for idx,i in enumerate(cmat.columns):
        N = 0
        for j,n in enumerate(i.split('_')):
            N+=(10**-j)*int(n)
        column_order.append(N)
    column_order = np.array(cmat.columns)[np.argsort(np.array(column_order))]
    cmat  = cmat[column_order].loc[idx_order]
    plt.figure(figsize=[15,15])
    p = sns.clustermap(cmat.T,cmap='jet',vmin=0,vmax=1,col_cluster=False)

In [ ]:
measured_data

In [ ]:
rc.data

In [ ]:
for level in rc.data.obsm:
    if 'updated' in level:
        print(level.split('_updated')[0])
        measured_data.obs[level.split('_updated')[0]] = np.array(rc.data.obsm[level])
        measured_data.obsm[level.split('_updated')[0]] = np.array(rc.data.obsm[level])

In [ ]:
# measured_data.write(filename=os.path.join(measured_path,'labeled_renorm_noHarmony_March18.h5ad'))
# measured_data

measured_data.obs.to_csv(os.path.join(measured_path,'labeled_renorm_zscore_Harmony_March23.csv'),
                         header=True, index=True
                        )